In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

max_words = 100
batch_sizes = 128
# 載入 imdb 資料集
(train_data, train_labels), (test_data, test_labels) = \
    imdb.load_data(num_words=10000)
print(len(train_data))  # 印出訓練集的筆數
print(len(test_data))   # 印出測試集的筆數
# 讓所有的影評資料保持在 100個字
train_data_new = sequence.pad_sequences(train_data,maxlen=max_words)
test_data_new = sequence.pad_sequences(test_data,maxlen=max_words)
db_train = tf.data.Dataset.from_tensor_slices((train_data_new,train_labels))
# 參數 drop_remainder = true 代表當最後一批少於 batch_size元素的情况下就刪除
# 將訓練資料打散
db_train = db_train.shuffle(1000).batch(batch_sizes,drop_remainder=True)
db_test = tf.data.Dataset.from_tensor_slices((test_data_new,train_labels))
db_test = db_test.batch(batch_sizes,drop_remainder=True)

25000
25000


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

vocab_size = 10000
model = Sequential()
model.add(layers.Embedding( vocab_size,
                            output_dim=120,
                            input_length= max_words))
# 除了最上面那一層之外, 其他層的輸出都必須做為下一層的輸入
model.add(layers.SimpleRNN(units=64,return_sequences=True,
                           dropout=0.25))
model.add(layers.SimpleRNN(units=64,dropout=0.25))
model.add(layers.Dense(units=1,activation='sigmoid'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 120)          1200000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 100, 64)           11840     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                8256      
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1,220,161
Trainable params: 1,220,161
Non-trainable params: 0
_________________________________________________________________
None


In [3]:
model.compile(loss='binary_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])
history = model.fit(db_train,batch_size=batch_sizes,
                    epochs=10,verbose=2)

Epoch 1/10


2022-10-28 16:21:54.865714: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


195/195 - 17s - loss: 0.5400 - accuracy: 0.6919 - 17s/epoch - 86ms/step
Epoch 2/10
195/195 - 16s - loss: 0.3164 - accuracy: 0.8691 - 16s/epoch - 83ms/step
Epoch 3/10
195/195 - 16s - loss: 0.2124 - accuracy: 0.9193 - 16s/epoch - 81ms/step
Epoch 4/10
195/195 - 17s - loss: 0.1289 - accuracy: 0.9536 - 17s/epoch - 85ms/step
Epoch 5/10
195/195 - 16s - loss: 0.1019 - accuracy: 0.9622 - 16s/epoch - 83ms/step
Epoch 6/10
195/195 - 16s - loss: 0.0587 - accuracy: 0.9804 - 16s/epoch - 82ms/step
Epoch 7/10
195/195 - 16s - loss: 0.0542 - accuracy: 0.9803 - 16s/epoch - 83ms/step
Epoch 8/10
195/195 - 16s - loss: 0.0293 - accuracy: 0.9901 - 16s/epoch - 83ms/step
Epoch 9/10
195/195 - 16s - loss: 0.0298 - accuracy: 0.9894 - 16s/epoch - 83ms/step
Epoch 10/10
195/195 - 16s - loss: 0.0263 - accuracy: 0.9910 - 16s/epoch - 83ms/step
